In [1]:
import torch 
from models import Generator , MolGANDiscriminator
from utils import MolecularMetrics
from sparse_molecular_dataset import SparseMolecularDataset
from tools import postprocess,get_reward,sample_z,label2onehot,reward,get_gen_mols,save_mol_img
from collections import defaultdict
import torch.nn as nn


In [2]:
G = Generator()
D = MolGANDiscriminator()
R = MolGANDiscriminator()   

g_optim = torch.optim.Adam(G.parameters(),lr = 1e-3,betas=(0.0,0.9))
d_optim = torch.optim.Adam(D.parameters(),lr = 1e-3,betas=(0.0,0.9))
v_optim = torch.optim.Adam(R.parameters(),lr = 1e-3,betas=(0.0,0.9))


In [3]:
data = SparseMolecularDataset()


In [6]:
class Solve:
    def __init__(self,batch_size = 32):
        self.batch_size = batch_size
        self.data = data
        self.data.load("qm9_5k.sparsedataset")
        self.m_dim = self.data.atom_num_types
        self.b_dim = self.data.bond_num_types
        self.num_steps = len(data)//self.batch_size
        self.lamd = 0.5
        self.alpha = 10
        self.n_critic = 5
        # ADDED: A loss function is required for the reward network.
        self.loss_fn = nn.MSELoss()
        
    def train_or_valid(self, epoch_i, total_epochs, train_val_test="train"):
            # ==========================================================
            # FIXED: Correct training and pre-training schedule
            # ==========================================================
            # As per the paper, the generator trains with WGAN loss only for the
            # first half of epochs (lambda=1.0) and a combined loss for the second half.
            if epoch_i < total_epochs / 2:
                self.lamd = 1.0  # Use WGAN loss only for the generator
            else:
                self.lamd = 0.5  # Use combined WGAN + RL loss

            losses = defaultdict(list)
            scores = defaultdict(list)
            the_step = self.num_steps
            for a_step in range(the_step):
                cur_step = self.num_steps * epoch_i + a_step

                if train_val_test == 'val':
                    mols, _, _, a, x, _, _, _, _ = self.data.next_validation_batch()
                    z =  sample_z(a.shape[0])
                elif train_val_test == 'train':
                    # --- Batch Loading (Unchanged) ---
                    mols, _, _, a, x, _, _, _, _ = self.data.next_train_batch(self.batch_size)
                    batch_dim = self.batch_size
                    z = sample_z(self.batch_size)

                a = torch.from_numpy(a).long()  
                x = torch.from_numpy(x).long()  
                a_tensor = label2onehot(a, self.b_dim)
                x_tensor = label2onehot(x, self.m_dim)
                z = torch.from_numpy(z).float()
                
                # --- Discriminator Training (Unchanged) ---
                d_optim.zero_grad()
                real_logits = D(x_tensor,a_tensor)
                fake_adjancency , fake_nodes = G(z)
                (adj_hat,nodes_hat) = postprocess((fake_adjancency , fake_nodes))
                fake_logits = D(nodes_hat.detach(), adj_hat.detach())
                
                d_loss_wgan = torch.mean(fake_logits) - torch.mean(real_logits)

                eps = torch.rand(batch_dim, 1, 1, 1).to(a_tensor.device)
                a_hat_grad = (eps * a_tensor + (1 - eps) * fake_adjancency.detach()).requires_grad_(True)
                x_hat_grad = (eps.squeeze(-1) * x_tensor + (1 - eps.squeeze(-1)) * fake_nodes.detach()).requires_grad_(True)
        
                logits_hat = D(x_hat_grad, a_hat_grad)
                gradients = torch.autograd.grad(
                    outputs=logits_hat,
                    inputs=(x_hat_grad, a_hat_grad),
                    grad_outputs=torch.ones(logits_hat.size()).to(logits_hat.device),
                    create_graph=True, retain_graph=True,
                )
                grad_flat = torch.cat((gradients[0].view(batch_dim, -1), gradients[1].view(batch_dim, -1)), dim=1)
                gradient_penalty = self.alpha * ((grad_flat.norm(2, dim=1) - 1) ** 2).mean()

                d_loss = d_loss_wgan + gradient_penalty
                d_loss.backward()
                d_optim.step()
                losses['d_loss'].append(d_loss.item())

                # --- Reward Network Training (Unchanged) ---
                if cur_step % self.n_critic == 0:
                    v_optim.zero_grad()
                    real_reward = R(x_tensor,a_tensor,reward = True)
                    fake_reward = R(nodes_hat.detach(),adj_hat.detach(),reward =True)
                    
                    reward_r = torch.from_numpy(reward(mols)).float().to(real_reward.device)
                    reward_f = get_reward(nodes_hat.detach(),adj_hat.detach()).float().to(fake_reward.device)
                                   
                    loss_V = self.loss_fn(real_reward, reward_r) + self.loss_fn(fake_reward, reward_f)
                    loss_V.backward()
                    v_optim.step()
                
                # --- Generator Training (Unchanged) ---
                g_optim.zero_grad()
                
                fake_logits_for_g = D(nodes_hat, adj_hat)
                g_loss_wgan = -torch.mean(fake_logits_for_g)
                
                fake_reward_for_g = R(nodes_hat, adj_hat, reward=True)
                g_loss_rl = -torch.mean(fake_reward_for_g)
                
                g_loss = self.lamd * g_loss_wgan + (1 - self.lamd) *g_loss_rl 
                g_loss.backward()
                g_optim.step()
                losses['g_loss'].append(g_loss.item())
                
                print(f"Epoch {epoch_i} Losses -> D: {d_loss.item():.4f}, G: {g_loss.item():.4f}")

In [16]:
solver = Solve()

# 4. Define Training Hyperparameters
total_epochs = 350 # As used in the paper for the 5k dataset

# ==========================================================
# 5. The Main Training Loop
# ==========================================================
print("Starting MolGAN training...")
for epoch_i in range(total_epochs):
    # Call the training method for the current epoch.
    # The 'train_val_test' argument defaults to "train".
    # We pass 'total_epochs' so the pre-training schedule inside the method works correctly.
    solver.train_or_valid(epoch_i, total_epochs)

print("Training finished.")


Starting MolGAN training...


[12:23:44] Explicit valence for atom # 0 N, 8, is greater than permitted
[12:23:44] Explicit valence for atom # 2 F, 8, is greater than permitted
[12:23:44] Explicit valence for atom # 0 N, 12, is greater than permitted
[12:23:44] Explicit valence for atom # 1 F, 14, is greater than permitted
[12:23:44] Explicit valence for atom # 2 F, 10, is greater than permitted
[12:23:44] Explicit valence for atom # 1 N, 10, is greater than permitted
[12:23:44] Explicit valence for atom # 0 C, 13, is greater than permitted
[12:23:44] Explicit valence for atom # 1 F, 14, is greater than permitted
[12:23:44] Explicit valence for atom # 1 N, 12, is greater than permitted
[12:23:44] Explicit valence for atom # 0 C, 10, is greater than permitted
[12:23:44] Explicit valence for atom # 0 C, 13, is greater than permitted
[12:23:44] Explicit valence for atom # 0 N, 15, is greater than permitted
[12:23:44] Explicit valence for atom # 0 C, 15, is greater than permitted
[12:23:44] Explicit valence for atom # 0

Epoch 0 Losses -> D: 7.9881, G: 0.0338


[12:23:45] Explicit valence for atom # 1 F, 10, is greater than permitted
[12:23:45] Explicit valence for atom # 0 N, 13, is greater than permitted
[12:23:45] Explicit valence for atom # 0 N, 16, is greater than permitted
[12:23:45] Explicit valence for atom # 0 F, 8, is greater than permitted
[12:23:45] Explicit valence for atom # 0 N, 10, is greater than permitted
[12:23:45] Explicit valence for atom # 0 O, 11, is greater than permitted
[12:23:45] Explicit valence for atom # 0 O, 9, is greater than permitted
[12:23:45] Explicit valence for atom # 1 C, 12, is greater than permitted
[12:23:45] Explicit valence for atom # 0 F, 13, is greater than permitted
[12:23:45] Explicit valence for atom # 0 N, 11, is greater than permitted
[12:23:45] Explicit valence for atom # 0 N, 12, is greater than permitted
[12:23:45] Explicit valence for atom # 0 C, 10, is greater than permitted
[12:23:45] Explicit valence for atom # 0 C, 13, is greater than permitted
[12:23:45] Explicit valence for atom # 0

Epoch 1 Losses -> D: 6.2460, G: 0.0399


...

Epoch 237 Losses -> D: -71.0419, G: 18.3852


[12:24:32] Explicit valence for atom # 0 C, 17, is greater than permitted
[12:24:32] Explicit valence for atom # 1 F, 17, is greater than permitted
[12:24:32] Explicit valence for atom # 0 N, 13, is greater than permitted
[12:24:32] Explicit valence for atom # 0 C, 11, is greater than permitted
[12:24:32] Explicit valence for atom # 3 N, 11, is greater than permitted
[12:24:32] Explicit valence for atom # 0 O, 7, is greater than permitted
[12:24:32] Explicit valence for atom # 1 F, 12, is greater than permitted
[12:24:32] Explicit valence for atom # 0 O, 10, is greater than permitted
[12:24:32] Explicit valence for atom # 1 F, 14, is greater than permitted
[12:24:32] Explicit valence for atom # 0 F, 11, is greater than permitted
[12:24:32] Explicit valence for atom # 1 O, 11, is greater than permitted
[12:24:32] Explicit valence for atom # 0 C, 15, is greater than permitted
[12:24:32] Explicit valence for atom # 1 O, 10, is greater than permitted
[12:24:32] Explicit valence for atom # 

Epoch 238 Losses -> D: -70.9947, G: 18.3517
Epoch 239 Losses -> D: -71.8564, G: 17.7393


[12:24:32] Explicit valence for atom # 1 N, 17, is greater than permitted
[12:24:32] Explicit valence for atom # 1 O, 9, is greater than permitted
[12:24:32] Explicit valence for atom # 2 C, 10, is greater than permitted
[12:24:32] Explicit valence for atom # 0 N, 8, is greater than permitted
[12:24:32] Explicit valence for atom # 0 N, 11, is greater than permitted
[12:24:32] Explicit valence for atom # 0 C, 10, is greater than permitted
[12:24:32] Explicit valence for atom # 0 N, 9, is greater than permitted
[12:24:32] Explicit valence for atom # 0 O, 12, is greater than permitted
[12:24:32] Explicit valence for atom # 0 C, 13, is greater than permitted
[12:24:32] Explicit valence for atom # 0 N, 14, is greater than permitted
[12:24:32] Explicit valence for atom # 0 C, 11, is greater than permitted
[12:24:32] Explicit valence for atom # 0 N, 15, is greater than permitted
[12:24:32] Explicit valence for atom # 0 C, 10, is greater than permitted
[12:24:32] Explicit valence for atom # 0 

KeyboardInterrupt: 

In [12]:
G.eval()
with torch.no_grad():
    z = torch.randn(100,32)
    adj_raw, nodes_raw = G(z)
    mols = get_gen_mols(nodes_raw,adj_raw)
    save_mol_img(mols,"mol.png")
    
    
    

[12:21:41] Explicit valence for atom # 1 O, 13, is greater than permitted
[12:21:41] Explicit valence for atom # 0 O, 10, is greater than permitted
[12:21:41] Explicit valence for atom # 1 F, 10, is greater than permitted
[12:21:41] Explicit valence for atom # 0 F, 14, is greater than permitted
[12:21:41] Explicit valence for atom # 1 F, 16, is greater than permitted
[12:21:41] Explicit valence for atom # 0 N, 13, is greater than permitted
[12:21:41] Explicit valence for atom # 0 N, 10, is greater than permitted
[12:21:41] Explicit valence for atom # 0 N, 15, is greater than permitted
[12:21:41] Explicit valence for atom # 1 C, 11, is greater than permitted
[12:21:41] Explicit valence for atom # 0 N, 9, is greater than permitted
[12:21:41] Explicit valence for atom # 0 C, 12, is greater than permitted
[12:21:41] Explicit valence for atom # 0 N, 15, is greater than permitted
[12:21:41] Explicit valence for atom # 1 C, 12, is greater than permitted
[12:21:41] Explicit valence for atom # 